In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from numpy import zeros_like
from IPython.display import display
def display_matrix(m):
    display(sympy.Matrix(m))
import sympy
import pandas as pd
import scipy.stats as ss

sympy.init_printing()

from Pricer import Pricer
from Processes import Processes
from Processes1 import Processes1
from Processes2 import Processes2

from scipy.stats.qmc import Sobol
from scipy.stats import norm

In [2]:
basis = 'leguerre'

# parameters

mu = 0.06
r = 0.06
T = 1
I = 250
paths = 1000
S0 = 40
S0_list = range(37, 43, 1)
m = 0
K = 40
sigma = 0.04

GBM_sigma = 0.04
GBM_mu = r

LN_lam = 1
LN_sigma = 0.02
LN_mu = r
LN_v = 0.02

JR_lam = 0.01
JR_sigma = 0.03
JR_mu = r + JR_lam

LNparams = (LN_lam, LN_sigma, LN_mu, LN_v, m)
JRparams = (JR_lam, JR_sigma, JR_mu)
GBMparams = (GBM_mu, GBM_sigma)

# instance
processes = Processes(S0, r, T, K, paths, I, LNparams, JRparams, GBMparams)
pricer = Pricer(S0, r, sigma, T, K, paths, I, LNparams, JRparams, GBMparams)
processes1 = Processes1(S0, r, T, K, paths, I, LNparams, JRparams, GBMparams)
processes2 = Processes2(S0, r, T, K, paths, I, LNparams, JRparams, GBMparams)

Processes class created with S0 = 40, paths = 1000, I = 250, T = 1
Processes1 class created with S0 = 40, paths = 1000, I = 250, T = 1
Processes2 class created with S0 = 40, paths = 1000, I = 250, T = 1


In [8]:
# Containers for storing matrices
GBM_matrices = {}
LN_matrices = {}
JR_matrices = {}

GBM_matrices_antithetic = {}
LN_matrices_antithetic = {}
JR_matrices_antithetic = {}

# Containers for storing prices
GBM_prices = []
LN_prices = []
JR_prices = []
GBM_prices_antithetic = []
LN_prices_antithetic = []

GBM_SE = []
LN_SE = []
JR_SE = []
GBM_SE_antithetic = []
LN_SE_antithetic = []

# Generate matrices and calculate prices for each S0 in S0_list
for S0 in S0_list:
    processes.S0 = S0
    GBM_matrices[S0] = processes.gbm_paths()
    LN_matrices[S0] = processes.merton_jump_paths()
    JR_matrices[S0] = processes.merton_jump_to_ruin_paths()
    GBM_matrices_antithetic[S0] = processes.gbm_paths_antithetic()
    LN_matrices_antithetic[S0] = processes.merton_jump_paths_antithetic()
    
    GBM_prices.append(pricer.LSM(GBM_matrices[S0], basis, 3)[0])
    LN_prices.append(pricer.LSM(LN_matrices[S0], basis, 3)[0])
    JR_prices.append(pricer.LSM(JR_matrices[S0], basis, 3)[0])
    GBM_prices_antithetic.append(pricer.LSM(GBM_matrices_antithetic[S0], basis, 3)[0])
    LN_prices_antithetic.append(pricer.LSM(LN_matrices_antithetic[S0], basis, 3)[0])
    
    GBM_SE.append(pricer.LSM(GBM_matrices[S0], basis, 3)[2])
    LN_SE.append(pricer.LSM(LN_matrices[S0], basis, 3)[2])
    JR_SE.append(pricer.LSM(JR_matrices[S0], basis, 3)[2])
    GBM_SE_antithetic.append(pricer.LSM(GBM_matrices_antithetic[S0], basis, 3)[2])
    LN_SE_antithetic.append(pricer.LSM(LN_matrices_antithetic[S0], basis, 3)[2])

/Users/philipgadereumert/opt/anaconda3/lib/python3.9/site-packages/numpy/polynomial/hermite.py:1404: RankWarning: The fit may be poorly conditioned
  return pu._fit(hermvander, x, y, deg, rcond, full, w)


In [12]:
# Print the V0 price for each model and each S0
for i, S0 in enumerate(S0_list):
    print ("S0 = %s" % S0)
    print ("GBM V0 = %s" % GBM_prices[i].round(3))
    print ("LN V0 = %s" % LN_prices[i].round(3))
    print ("JR V0 = %s" % JR_prices[i].round(3))
    print ("GBM V0 antithetic = %s" % GBM_prices_antithetic[i].round(3))
    print ("LN V0 antithetic = %s" % LN_prices_antithetic[i].round(3))
    print ("")

S0 = 37
GBM V0 = 2.932
LN V0 = 2.947
JR V0 = 2.933
GBM V0 antithetic = 2.938
LN V0 antithetic = 2.938

S0 = 38
GBM V0 = 1.953
LN V0 = 1.946
JR V0 = 2.058
GBM V0 antithetic = 1.957
LN V0 antithetic = 1.96

S0 = 39
GBM V0 = 0.97
LN V0 = 0.97
JR V0 = 0.977
GBM V0 antithetic = 0.977
LN V0 antithetic = 0.978

S0 = 40
GBM V0 = 0.182
LN V0 = 0.173
JR V0 = 0.294
GBM V0 antithetic = 0.159
LN V0 antithetic = 0.129

S0 = 41
GBM V0 = 0.018
LN V0 = 0.05
JR V0 = 0.042
GBM V0 antithetic = 0.018
LN V0 antithetic = 0.065

S0 = 42
GBM V0 = 0.004
LN V0 = 0.023
JR V0 = 0.007
GBM V0 antithetic = 0.001
LN V0 antithetic = 0.014



In [10]:
# print the standard error for each model and each S0
for i, S0 in enumerate(S0_list):
    print ("S0 = %s" % S0)
    print ("GBM SE = %s" % GBM_SE[i].round(3))
    print ("LN SE = %s" % LN_SE[i].round(3))
    print ("JR SE = %s" % JR_SE[i].round(3))
    print ("GBM SE antithetic = %s" % GBM_SE_antithetic[i].round(3))
    print ("LN SE antithetic = %s" % LN_SE_antithetic[i].round(3))
    print ("")

S0 = 37
GBM SE = 0.006
LN SE = 0.01
JR SE = 0.008
GBM SE antithetic = 0.004
LN SE antithetic = 0.002

S0 = 38
GBM SE = 0.007
LN SE = 0.01
JR SE = 0.142
GBM SE antithetic = 0.005
LN SE antithetic = 0.011

S0 = 39
GBM SE = 0.007
LN SE = 0.003
JR SE = 0.009
GBM SE antithetic = 0.004
LN SE antithetic = 0.005

S0 = 40
GBM SE = 0.011
LN SE = 0.028
JR SE = 0.085
GBM SE antithetic = 0.008
LN SE antithetic = 0.017

S0 = 41
GBM SE = 0.004
LN SE = 0.015
JR SE = 0.018
GBM SE antithetic = 0.002
LN SE antithetic = 0.014

S0 = 42
GBM SE = 0.002
LN SE = 0.007
JR SE = 0.002
GBM SE antithetic = 0.0
LN SE antithetic = 0.005

